# Projet phylogénétique
Marine Djaffardjy (marine.djaffardjy@u-psud.fr)- Théophile Sanchez - Sarah Cohen Boulakia

------

## Introduction

Au cours de ce projet, vous étudierez trois espèces disparues de félins qui vivaient autrefois sur le continent Américain. Ces trois espèces, le _smilodon_ (tigre à dents de sabre), l'_homotherium_ (_scimitar toothed tigers_) et _M. trumani_ (guépard américain) se sont éteintes il y a environ 13 000 ans, à la fin de la dernière période glaciaire. Des séquences ADN partielles de la protéine cytochrome b de ces espèces ont pu être séquencées et vont vous permettre de retrouver les liens de parentés entre ces espèces et des espèces de félins contemporaines : le chat domestique, le lion, le léopard, le tigre, le puma, le guépard et les chats sauvages africains, chinois et européens. Sont aussi présentes dans le jeu de donnée des séquences issues d'espèces extérieures aux félins.

Afin de reconstruire l'arbre phylogénétique de ces espèces, vous utiliserez une méthode basée sur le calcul des distances évolutives entre les séquences ADN des protéines. Sachez qu'une démarche similaire peut-être appliquée aux séquences d'acides aminés.

Les différentes étapes qui vous permetterons de construire l'arbre sont détaillées dans ce notebook. Vous devrez implémenter les algorithmes en Python et répondre aux questions dans les cellules prévues.

Quelques conseils :
- Utiliser au maximum les fonctions présentes dans les packages de python (sauf si il vous est explicitement demandé de les réimplémenter). Si un problème vous paraît courant, il existe surement déjà une fonction pour le résoudre. Pour ce projet vous serez limités aux packages de base, à Numpy et ETE (seulement pour l'affichage des arbres). Vous n'avez pas besoin d'utiliser d'autre packages (pas de pandas).
- Si une partie de votre code ne vous semble pas très explicite, ajoutez des commentaires pour l'expliquer. Une personne qui lit votre code doit pouvoir comprendre son fonctionnement facilement.
- Faites des fonctions d'affichage pour vos matrices !!
- N'hésitez pas à chercher dans la documentation et sur internet. Cependant, faites attention au plagiat !

Le projet est à rendre **en binôme** par mail. Vous regrouperez votre notebook et les fichiers nécessaires à son fonctionnement dans une archive portant vos noms et prénoms (il faut que je puisse faire tourner les notebooks).

------
## Importation des séquences

Le format [FASTA](https://fr.wikipedia.org/wiki/FASTA_(format_de_fichier) permet de stocker plusieurs séquences (ADN, ARN ou peptidiques) dans un fichier. Les séquences que vous allez étudier ont été regroupées dans le fichier `cat_dna.fasta`.

**Exercice 1 :** Écrivez une fonction permettant d'importer un fichier au format fasta et de le stocker dans un dictionnaire `seq_cat`. Les clés seront les noms des séquences et les valeurs du dictionnaire seront les séquences d'adn.
Exportez le dictionnaire obtenu dans un fichier `output/dict_seq.json`.

In [112]:
# À remplir
import json

def lire(path):
    seq_cat = {}
    fichier = open(path, "r")
    lignes = fichier.readlines()
    for ligne in lignes :
        if ligne.startswith(">"):
            id = ligne[1:].split()[0]
            seq_cat[id] = ''
        else :
            seq_cat[id] += ligne.strip()
    with open('output/dict_seq.json', 'w') as mon_fichier:
	    json.dump(seq_cat, mon_fichier)

lire("input/cat_dna.fasta")


------
## Alignement des séquences

La méthode que vous utiliserez pour calculer l'arbre phylogénétique nécessite de calculer la distance évolutive entre les séquences. Avant de pouvoir les calculer, il faut d'abord aligner les séquences en considérant trois types de mutations :
- les substitutions (un nucléotide est remplacé par un autre)
- les insertions
- les délétions
Par exemple, les séquences "ACTCCTGA" et "ATCTCGTGA" ont plusieurs alignements possibles : 

$A_1$ :
```
-ACTCCTGA
ATCTCGTGA
```

$A_2$ :
```
A-CTCCTGA
ATCTCGTGA
```

$A_3$ :
```
AC-TCCTGA
ATCTCGTGA
```
.

.

.

Le "-" désigne un *gap*, c'est à dire un "trou" dans l'alignement qui a été causé par une insertion ou une déletion. On regroupe ces deux types de mutations sous le terme indel.

Ces alignements correspondent à une multitude d'histoires phylogénétiques différentes. Pour sélectionner le meilleur alignement il faut donc introduire l'hypothèse du maximum de parcimonie qui privilégie l'histoire phylogénétique qui implique le moins d'hypothèses et donc, le moins de changements évolutifs. Par exemple, parmis les trois alignements ci-dessus on preferera l'alignement 2 car il correspond au scénario avec le moins de mutations:
- l'alignement 1 implique au minimum 1 indel et 2 substitutions
- l'alignement 2 implique au minimum 1 indel et 1 substitutions
- l'alignement 3 implique au minimum 1 indel et 2 substitutions

On peut maintenant définir un score d'identité que l'on va augmenter de 1 lorsque qu'il n'y pas eu de mutation et ainsi obtenir la matrice suivante :

|   &nbsp;   | A | C | G | T | - |
|   -   | - | - | - | - | - |
| **A** | 1 | 0 | 0 | 0 | 0 |
| **C** | 0 | 1 | 0 | 0 | 0 |
| **G** | 0 | 0 | 1 | 0 | 0 |
| **T** | 0 | 0 | 0 | 1 | 0 |
| **-** | 0 | 0 | 0 | 0 | 0 |

Cette matrice correspond au modèle d'évolution de l'ADN défini par Jukes et Cantor qui fait l'hypothèse d'un taux de mutation équivalent pour chacun des nucléotides. Cependant, en réalité ces taux ne sont pas les mêmes partout, on sait par exemple que le taux de transition (substitution A$\leftrightarrow$G ou C$\leftrightarrow$T) est différent du taux de transversions (substitution A$\leftrightarrow$T, C$\leftrightarrow$G, C$\leftrightarrow$A ou G$\leftrightarrow$T) et que d'autres facteurs devrait être pris en compte comme la fréquence du nucléotide dans l'ADN. [C'est pour cette raison qu'il existe beaucoup de modèles différents d'écrivant l'évolution de l'ADN.](https://en.wikipedia.org/wiki/Models_of_DNA_evolution) Dans la suite de ce projet nous utiliserons la matrice de similarité $S$ suivante : 

|   &nbsp;   | A  | C  | G  | T  | -  |
|   -   | -  | -  | -  | -  | -  |
| **A** | 10 | -1 | -3 | -4 | -5 |
| **C** | -1 | 7  | -5 | -3 | -5 |
| **G** | -3 | -5 | 9  | 0  | -5 |
| **T** | -4 | -3 | 0  | 8  | -5 |
| **-** | -5 | -5 | -5 | -5 | 0  |

**Exercice 2 :** Écriver la fonction permettant de calculer le score entre deux alignements avec la matrice de similarité précédente puis afficher le score des trois alignements $A_1$, $A_2$ et $A_3$. La classe permettant d'importer une matrice et de calculer le score entre deux lettres vous est déjà fournie, la matrice de similarité est stockée dans le fichier `dna_matrix` :


In [2]:
import numpy as np


class SimilarityMatrix:
    def __init__(self, filename):
        with open(filename) as f:
            self.letters = f.readline().split()
            self.values = np.loadtxt(filename, skiprows=1, usecols=range(1, len(self.letters) + 1))
        
    def score(self, letter1, letter2): # return the similarity score between letter1 and letter2
        return self.values[self.letters.index(letter1)][self.letters.index(letter2)]
    
# Example
similarity_matrix = SimilarityMatrix('input/dna_matrix.txt')
print('Score between G and C:', similarity_matrix.score('G', 'C'))
print('Score between A and a gap:', similarity_matrix.score('-', 'A'))

Score between G and C: -5.0
Score between A and a gap: -5.0


In [3]:
def score2(seq1, seq2) : 
    score = 0
    for i in range(len(seq2)):
        score +=  similarity_matrix.score(seq1[i],seq2[i])
    return score


A = ['-ACTCCTGA', 'A-CTCCTGA', 'AC-TCCTGA']
B = 'ATCTCGTGA'
for i in range(3):
    print(f'Le score de l\'alignement A{i+1} est de {score2(A[i], B)}')
        

Le score de l'alignement A1 est de 35.0
Le score de l'alignement A2 est de 49.0
Le score de l'alignement A3 est de 39.0


------
### Algorithme de Needleman-Wunsch

Maintenant que vous avez vu ce qu'est une matrice de similarité et comment calculer le score de similarité d'un alignement, vous allez devoir implémenter un algorithme permettant de trouver le meilleur alignement global entre deux séquences. Avec deux séquences à aligner de taille $n$ et $m$, la première étape consiste à initialiser deux matrices de taille $(n+1) \times (m+1)$. La première est la matrice de score $M$ et la seconde sera la matrice de *traceback* $T$. 

Par exemple, avec la matrice $S$ et les séquences $A =$ "ACTCCTGA" et $B =$ "ATCTCGTGA", on initialise $M$ comme si l'on ajoutait des *gaps* partout :

|   &nbsp;   | - | A | T | C | T | C | G | T | G | A |
|   -   | - | - | - | - | - | - | - | - | - | - |
| **-** | 0 |-5 |-10|-15|-20|-25|-30|-35|-40|-45|
| **A** |-5 | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **C** |-10| &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **T** |-15| &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **C** |-20| &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **C** |-25| &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **T** |-30| &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **G** |-45| &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **A** |-40| &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; ||

Puis on initialise $T$ :

|   &nbsp;   | - | A | T | C | T | C | G | T | G | A |
|   -   | - | - | - | - | - | - | - | - | - | - |
| **-** | o | l | l | l | l | l | l | l | l | l |
| **A** | u | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **C** | u | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **T** | u | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **C** | u | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **C** | u | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **T** | u | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **G** | u | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **A** | u | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; ||


Il faut ensuite remplir la matrice $M$ en suivant la formule $M_{ij} = \max(M_{i-1j-1} + s(A_i, B_j), M_{ij-1} + s(A_i, gap), M_{i-1j} + s(B_j,gap) )$ avec $i \in {2, \dots, n+1}$, $j \in {2, \dots, m+1}$ et la fonction $s$ qui calcule le score de similarité entre deux nucléotides. Pour chaque case de $T$ on remplie par :
- 'd' (*diagonal*) si $M_{ij}$ a été calculé en utilisant la diagonale $M_{i-1j-1}$,
- 'l' (*left*) si $M_{ij}$ a été calculé en utilisant la case de gauche $M_{ij-1}$,
- 'u' (*up*) si $M_{ij}$ a été calculé en utilisant la case du haut $M_{i-1j}$.

On obtient alors les matrices suivantes $M$ et $T$ : 

|   &nbsp;   | - | A | T | C | T | C | G | T | G | A |
|   -   | - | - | - | - | - | - | - | - | - | - |
| **-** |  0| -5|-10|-15|-20|-25|-30|-35|-40|-45|
| **A** | -5| 10|  5|  0| -5|-10|-15|-20|-25|-30|
| **C** |-10|  5|  7| 12|  7|  2| -3| -8|-13|-18|
| **T** |-15|  0| 13|  8| 20| 15| 10|  5|  0| -5|
| **C** |-20| -5|  8| 20| 15| 27| 22| 17| 12|  7|
| **C** |-25|-10|  3| 15| 17| 22| 22| 19| 14| 11|
| **T** |-30|-15| -2| 10| 23| 18| 22| 30| 25| 20|
| **G** |-35|-20| -7|  5| 18| 18| 27| 25| 39| 34|
| **A** |-40|-25|-12|  0| 13| 17| 22| 23| 34| 49|

|   &nbsp;   | - | A | T | C | T | C | G | T | G | A |
|   -   | - | - | - | - | - | - | - | - | - | - |
| **-** | o | l | l | l | l | l | l | l | l | l |
| **A** | u | d | l | l | l | l | l | l | l | d |
| **C** | u | u | d | d | l | d | l | l | l | l |
| **T** | u | u | d | l | d | l | l | d | l | l |
| **C** | u | u | u | d | l | d | l | l | l | l |
| **C** | u | u | u | d | d | d | d | d | l | d |
| **T** | u | u | d | u | d | l | d | d | l | l |
| **G** | u | u | u | u | u | d | d | u | d | l |
| **A** | u | d | u | u | u | d | u | d | u | d |

Il suffit maintenant de regarder le dernier élément $M_{nm} = 49$ pour avoir le score de l'alignement. Pour avoir l'alignement lui-même, il faut partir de $T_{nm}$ et remonter la "trace" jusqu'à arriver au 'o'. Un 'd' correspond à un *match* entre les deux séquences, 'l' à un *gap* dans la séquence $A$ et 'u' à un *gap* dans la séquence $B$. En revenant à l'exemple précédent on obtient la trace suivante :

|   &nbsp;   | - | A | T | C | T | C | G | T | G | A |
|   -   | - | - | - | - | - | - | - | - | - | - |
| **-** | o | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **A** | &nbsp; | d | l | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **C** | &nbsp; | &nbsp; | &nbsp; | d | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **T** | &nbsp; | &nbsp; | &nbsp; | &nbsp; | d | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **C** | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | d | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **C** | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | d | &nbsp; | &nbsp; | &nbsp; |
| **T** | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | d | &nbsp; | &nbsp; |
| **G** | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | d | &nbsp; |
| **A** | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | &nbsp; | d |

Elle correspond à l'alignement :
```
A-CTCCTGA
ATCTCGTGA
```

**Exercice 3 :** Implémenter l'algorithme de Needlman et Wunsch. Il prendra en paramètre deux séquences et une matrice de similarité et retournera leur alignement. Tester le avec les séquences "ACTCCTGA" et "ATCTCGTGA".

In [4]:
# À remplir
similarity_matrix = SimilarityMatrix('input/dna_matrix.txt')

def needWunch(A, B, similarity_matrix):
    #Etape 1 : initialisation de deux matrices de taille (n+1,m+1):
    #n étant la taille de la séquence A et m la taille de la séquence B
    #on dispose fictivement la séquence A en colonne et B en ligne
    #M: avec en première ligne et première colonne le score entre une nucléotide et un gap
    #T: avec un 'o' en T[0,0] et des 'l' en première ligne et 'u' en première colonne
    n = len(A)
    m = len(B)
    M = np.empty([n+1, m+1], dtype=int)
    T = np.empty([n+1, m+1], dtype=str)
    lA = ['o']+['l']*n
    lB = ['o']+['u']*m
    for j in range(m+1) :
        M[0, j] =  j * (-5)
        T[0, j] = lB[j]
    for i in range(n+1): 
        M[i, 0] = i * (-5)
        T[i, 0] = lA[i]
        
    #Etape 2: On remplit la matrice de score pour tout i dans [1,n] et j dans [1,m] en suivant la loi :
    # M[i,j] = max(M[i-1,j-1]+ similarity_matrix.score(A[i], B[j]), M[i, j-1]+ similarity_matrix.score(A[i],gap),   M[i-1,j]+ similarity_matrix.score(B[j], gap)
    # Et on remplit T suivant cet algorithme :
    #'d'  si M[i,j] a été calculé en utilisant la diagonale M[i-1,j-1],
    #'l'  si M[i,j] a été calculé en utilisant la diagonale M[i,j-1],
    #'u'  si M[i,j] a été calculé en utilisant la diagonale M[i-1,j]
    
    for i in range(1,n+1):
        for j in range(1,m+1):
            vald = M[i-1,j-1]+ similarity_matrix.score(A[i-1], B[j-1])
            vall = M[i, j-1]+ similarity_matrix.score(A[i-1],"-")
            valu =  M[i-1,j]+ similarity_matrix.score(B[j-1], "-")
            M[i,j] = max(vald, vall, valu)
            if M[i,j] == vald :
                T[i,j] = 'd'
            elif M[i,j] == vall :
                T[i,j] = 'l'
            elif M[i,j] == valu :
                T[i,j] = 'u'
            #print(M[i,j])
            #print(vald)
            #print(vall)
            #print(valu)
    
    #Etape 3 : Obtention de l'alignement
    #On démarre de la case T[n,m] jusqu à 'o'
    #Si la case courante de T correspond à un :
    # 'd': Les deux nucléotides de l'alignement sont identiques 
    # 'l' : Un gap dans la séquence A
    # 'u': Un gap dans la séquence B
    i = n
    j = m
    Al = []
    Bl = []
    while i>0 and j>0 :
        if T[i,j] == 'd':
            Al.append(A[i-1])
            Bl.append(B[j-1])
            i -= 1
            j-= 1
        elif T[i,j] == 'l':
            Al.append('-')
            Bl.append(B[j-1])
            j -= 1
        elif T[i,j] == 'u':
            Al.append(A[i-1])
            Bl.append('-')
            i-=1
    return ''.join(reversed(Al)), ''.join(reversed(Bl))
        
    
A = 'ACTCCTGA'
B = 'ATCTCGTGA'
needWunch(A, B, similarity_matrix)

('A-CTCCTGA', 'ATCTCGTGA')

----
## Matrice de distance

Dans le cas de séquences très proches, on estime que la distance évolutive réelle entre les séquences est proche de la p-distance qui est simplement le nombre de substitution dans l'alignement sur le nombre total de nucléotide. Pour simplifier, on ignore les positions alignées à des gaps. On applique ensuite la correction de Jukes-Cantor. Sa formule est $-(\frac{3}{4})\ln(1-(\frac{4}{3})\times \textit{p-distance})$.

**Exercice 4 :** Implémenter la fonction retournant la matrice de distance à partir d'un dictionnaire de séquences. 

In [5]:
# À remplir
def jukes_cantor(p) :
    return -(3/4)*np.log(1-(4/3)*p) 

def matrice_distance(dic) :
    sequences = list(dic.values())
    distance_matrix = np.zeros((len(sequences), len(sequences)))
    for i in range(len(sequences)) :
        for j in range (len(sequences)) :
            score = 0
            if i!=j :
                align2, align1 = needWunch(sequences[j], sequences[i], similarity_matrix)
                for k in range(len(align2)) :
                    if (align2[k] != align1[k] and align2[k]!= "-" and align1[k]!="-"):
                        score+=1 
                distance_matrix[i][j]= jukes_cantor(score/len(align2))
    return distance_matrix

In [6]:
dic = {"un": "ACTGA", "deux": "TATCG", "trois": "CTGTT"}
print(matrice_distance(dic))

[[ 0.         -0.          0.2326162 ]
 [-0.          0.          0.57160504]
 [ 0.2326162   0.57160504  0.        ]]


**Exercice 4 bis :** À quoi sert la correction de Jukes-Cantor ? Quelle hypothèse fait-elle ? 

Réponse :

------
## Construction d'un arbre avec UPGMA

Grâce aux mesures de distances entre les séquences, on peut maintenant de construire l'arbre phylogénétique des globines. Vous allez devoir pour cela implémenter l'algorithme UPGMA (*unweighted pair group method with arithmetic mean*) qui, malgré son nom compliqué, est l'une des méthodes les plus simples pour la construction d'arbre.

### Le format Newick

Le format Newick est l'un des formats utilisé en phylogénie pour représenter un arbre sous la forme d'une chaine de caractère. Le principe est simple, les groupes ayant la même racine sont écrit entre parenthèses et séparés par des virgules. Un groupe peut être soit une feuille de l'arbre (dans notre cas une séquence), soit un autre groupe. La longueur de la branche de chaque groupe est écrite après un double point et l'arbre est terminé par un point virgule. Pour afficher l'arbre on peut utiliser les fonction du package ETE : 

In [49]:
!pip install ete3

In [50]:
from ete3 import Tree, TreeStyle

newick_tree = '((A:2,(B:2,C:3):5):5,D:4);'
t = Tree(newick_tree)
ts = TreeStyle()
ts.show_branch_length = True
t.render('%%inline', w=183, units='mm', tree_style=ts)

TypeError: arguments did not match any overloaded call:
  QImage(): too many arguments
  QImage(size: QSize, format: QImage.Format): argument 1 has unexpected type 'float'
  QImage(width: int, height: int, format: QImage.Format): argument 1 has unexpected type 'float'
  QImage(data: bytes, width: int, height: int, format: QImage.Format): argument 1 has unexpected type 'float'
  QImage(data: PyQt5.sip.voidptr, width: int, height: int, format: QImage.Format): argument 1 has unexpected type 'float'
  QImage(data: bytes, width: int, height: int, bytesPerLine: int, format: QImage.Format): argument 1 has unexpected type 'float'
  QImage(data: PyQt5.sip.voidptr, width: int, height: int, bytesPerLine: int, format: QImage.Format): argument 1 has unexpected type 'float'
  QImage(xpm: List[str]): argument 1 has unexpected type 'float'
  QImage(fileName: str, format: typing.Optional[str] = None): argument 1 has unexpected type 'float'
  QImage(a0: QImage): argument 1 has unexpected type 'float'
  

**Exercice 5 :** Reécrivez l'arbre suivant au format Newick puis afficher-le. Les nombres correspondent aux longueurs des branches :
![](input/tree.png)

In [8]:
# À remplir

**Exercice 6 :** Expliquer la relation de parenté entre $A$, $B$ et $C$ ? Qu'elles sont les hypothèses qui pourraient expliquer ce type d'embranchement dans un arbre ? Donner une réponse détaillée.

Réponse : 

### UPGMA

L'algorithme UPGMA se base sur la matrice de distance entre les séquences. À chaque itération, les séquences avec la distance la plus faible sont regroupées puis une nouvelle matrice de distance est calculée avec le nouveau groupe. Cette étape est répétée jusqu'à n'avoir plus qu'un seul groupe. Par exemple, avec la matrice de distance entre les séquences $A$, $B$, $C$ et $D$ suivante :

|   &nbsp;   | A | B | C | D |
|   -   | - | - | - | - |
| **A** | &nbsp; | &nbsp; | &nbsp; | &nbsp; |
| **B** | 4 | &nbsp; | &nbsp; | &nbsp; |
| **C** | 8 | 8 | &nbsp; | &nbsp; |
| **D** | 2 | 4 | 8 | &nbsp; |

Les séquences $A$ et $D$ sont les plus proches ($distance(A,D)=2$). On les regroupe et on met à jour la matrice :

|   &nbsp;   | (A, D) | B | C |
|   -   | - | - | - |
| **(A, D)** | &nbsp; | &nbsp; | &nbsp; |
| **B** | 4 | &nbsp; | &nbsp; |
| **C** | 8 | 8 | &nbsp; | &nbsp; |

On regroupe maintenant $(A,D)$ et $B$ ($distance((A,D),B) = 4$) :

|   &nbsp;   | ((A, D), B) | C |
|   -   | - | - |
| **((A, D), B)** | &nbsp; | &nbsp; |
| **C** | 8 | &nbsp; |

Important : les nouvelles distances sont calculées en moyennant les distances entre les membres du nouveau groupe et des groupes non modifiés pondéré par le nombre d'UTOs dans chaque groupe. Avec $i$ et $j$ les deux membres du groupe nouvellement formé et k les groupes restant : $d_{ij,k} = \frac{n_id_{ik}}{n_i + n_j}+ \frac{n_jd_{jk}}{n_i + n_j}$. Par exemple avec la distance entre $((A, D), B)$ et $C$:

$distance(((A, D), B), C) = (distance((A, D), C)*2 + distance(B, C)) \mathbin{/} 3 = (8*2 + 8) \mathbin{/} 3 = 8 $.

L'arbre final écrit dans le format Newick est : $((A, D), B), C);$ 

Et avec les distances : $((A:1, D:1):1, B:2):2, C:4);$ 

**Exercice 7 :** Implémenter une version d'UPGMA qui calcule l'arbre au format Newick **avec les distances** puis appliquer votre algorithme aux données. 

In [122]:

# À remplir

def coord_triang(M,i,j):
    if j<i :
        return M[i][j]
    else :
        return M[j][i]
    
def id_from_label(labels, label):
    for i in range(len(labels)):
        if labels[i] == label :
            return i
        
def cherche_min_idmin(M) :
    dist_min = 10**5
    for i in range(len(M)):
            for j in range(len(M[i])):
                if M[i][j] != 0 and dist_min > M[i][j] :
                    dist_min = M[i][j]
                    id_groupe = (i,j)
    return dist_min, id_groupe


    
def UPGMA(M, labels):
    
    seq = ""
    
    
    groupes = []
    
    id_labels = [i for i in range(len(labels))]
    
    last_dist = 0
    
    id_groupe = (-1,-1)
    dist_min = 10**5
    
    
    
    while len(M)>=2 :
        
    
        dist_min, id_groupe = cherche_min_idmin(M)
        
                    
        idx, idy = id_groupe[0], id_groupe[1]
        
        if idx < idy :
            id_groupe = idx, idy
        else :
            id_groupe = idy, idx
            
            
        groupe = labels[id_groupe[0]],labels[id_groupe[1]]
        groupes.append(groupe)
        
        
        labels_new = labels.copy()
        
        labels_new.pop(id_groupe[0]) 
        labels_new.pop(id_groupe[1]-1)
        labels_new.insert(0,groupe)
        
        
        dict = {}
        dict[groupe[0]] = dist_min /2
        
        seqaux = ''
        
        
        if(len(groupe[0]) == 1) and len(groupe[1]) == 1:
            seqaux += '(' + groupe[0] + ':' + str(int(dist_min /2)) + ')'
            seqaux += ','
            seqaux += '(' + groupe[1] + ':' + str(int(dist_min /2)) + ')'
        else :
            seqaux += ':' + str(int(dist_min/2)-int(last_dist/2))
            seqaux += ','
            seqaux += '(' + groupe[1] +':'+ str(int(dist_min/2)) +')'
            
        seq +=  seqaux 
            
        last_dist = dist_min
        
    
        id_labels_new = id_labels.copy()
        
        id_labels_new.pop(id_groupe[0]) 
        id_labels_new.pop(id_groupe[1]-1)
        id_labels_new.insert(0,id_groupe)
        
        
        Mnew = [[]]
        
        dist_pondere = []
        
        for k in id_labels_new[1:] :
                i = id_groupe[0]
                j = id_groupe[1]
                labi = labels[i]
                labj = labels[j]
                ni = len(labi)
                nj = len(labj)
                dik = coord_triang(M,i,k)
                djk = coord_triang(M, j, k)
                dist_pondere.append( ((ni * dik)+(nj * djk))/(ni + nj) )
                
                
        for x in range(1,len(M)-1):
            l = []
            l.append(dist_pondere[x-1])
            for y in range(1,x):
                if x > y :
                    l.append(coord_triang(M,id_labels[x], id_labels[y]))
            Mnew .append(l)
            
        
                
        M = Mnew
        labels = labels_new
        id_labels = id_labels_new
        
    print(seq)
        
    

In [119]:
dic_seq = {"Sabertooth": "CTAATTAAAATTATCAACCACTCATTCATTGATTTACCCACCCCATCCAACATTTCAGCATGATGAAACTTCGGCTCCTTATTAGGAGTGTGCTTAATCTTACAAATCCTCACTGGCTTATTTCTAGCCATACATTATACACCAGATACAACAACCGCCTTCTCATCAGTTACCCACATTTGCCGTGATGTTAATTACGGCTGAATTATCCGATATATACACGCCAATGGAGCTTCCATATTCTTCATCTGCCTATATATACATGTAGGTCGAGCATATACTAC", "Homotherium": "CTAATTAAAATCATCAACCAATCATTCATTGACTTACCTACCCCCTCCAACATCTCAGCATGATGAAACTTCGGATCCCTACTAGGCATTTGCCTAATTCTTCAAATCCTCACAGGCTTATTCCTAGCCATACACTACACATCAGACACAACAACTGCTTTCTCATCAATCGCCCATATTTGCCGTGACGTAAATTATGGTTGAATTATCCGATATATACACGCCAATGGAGCCTCTATATTCTTCATCTGTCTATACCTACATGTAGCTCGAGAATTTATTAC", "American": "CTTATTAAAATCATTAATCACTCATTCATTGATCTACCCACCCCATCCAACATTTCAGCATGATGAAACTTCGGTTCCCTACTAGGGGTCTGCCTAATCCTACAAATCCTAACCGGCCTCTTCCTGGCTATACACTACACATCAGACACAATAACCGCCTTTTCATCAGTTACTCACATCTGTCGTGACGTCAATTACGGCTGAATTATTCGGTATATACACGCCAACGGAGCCTCCATATTCTTTATCTGCCTATACATGCACGTAGGGCGAGAATATATTAC", "Spotted": "CTCATTAAAATTATCAACAAATCATTCATTGACCTCCCCACCCCATCCAACATCTCGGCATGGTGAAATTTCGGGTCACTATTAGGAATCTGCTTAATCTTACAAATCCTGACAGGTCTATTCCTAGCCATACACTACACATCAGACACAACAACCGCCTTCTCATCAGTGACCCACATCTGCCGAGACGTAAACTACGGCTGAATCATCCGATACATACACGCCAACGGAGCTTCCATATTCTTCATCTGTCTATATATACATATCGGCCGAGAATATACTAC", "Striped": "CTCATTAAAATTGTCAACGAATCATTCATCGATCTCCCCACCCCATCCAACATCTCAGCATGATGAAACTTCGGATCGCTATTAGGAATCTGCCTAATCTTACAGATTCTGACAGGCCTATTTCTAGCCATACACTACACATCAGACACAACAACCGCCTTTTCATCAGTAACACACATCTGCCGAGACGTCAACTATGGCTGAATTATCCGATATATGCACGCCAACGGGGCCTCCATGTTTTTCATCTGCCTGTTCATGCACGCCGGCCGAGAATGTACTAC", "Chinese": "CTTATTAAAATCATCAACCATTCATTCATTGACCTACCCACCCCATCCAACATCTCAGCATGATGAAATTTCGGCTCCCTATTAGGAGTCTGCCTAATCTTACAAATTCTCACCGGCCTCTTTCTAGCCATACACTACACATCAGACACAGTAACCGCTTTTTCATCAGTTACTCACATCTGTCGCGATGTTAATTACGGCTGAATCATCCGATACATACACGCCAATGGAGCTTCCATATTCTTTATCTGCCTATACATGCACGTAGGACGAGAATATATTAC", "African": "CTTATCAAAATTATTAATCACTCATTCATCGATCTACCCGCCCCATCTAACATCTCAGCATGATGAAACTTCGGCTCCCTTCTAGGAGTCTGCCTAATCTTACAAATCCTCACCGGCCTCTTTTTGGCCATACACTACACATCAGACACAATAACCGCCTTTTCATCAGTTACCCACATCTGTCGCGACGTTAATTATGGCTGAATCATCCGATATTTACACGCCAACGGAGCTTCTATATTCTTTATCTGCCTGTACATACACGTAGGACGGGAATATACTAC", "Felis": "CTTACCAAAATTATTAATCACTCATTCATCGACCTACCTGCCCCATCTAACATCTCAGCATGATGAAACTTCGGCTCCCTTCTAGGAGTCTGCCTAATCTTACAAATCCTCACCGGCCTCTTTTTGGCCATACACTACACATCAGACACAACAACCGCCTTTTCATCAGTTACCCACATCTGTCGCGACGTTAATTATGGCTGAATCATCCGATATTTACACGCCAACGGAGCTTCTATATTCTTTATCTGCCTGTACATACATGTAGGACGGGAATATACTAC", "Wild": "CTTATCAAAATTATTAATCACTCATTCATCGACCTACCCGCCCCATCTAACATCTCAGCATGATGAAACTTCGGCTCCCTTCTAGGAGTCTGCCTAATCTTACAAATCCTCACCGGCCTCTTTTTGGCCATACACTACACATCAGACACAATAACCGCCTTTTCATCAGTTACCCACATCTGTCGCGACGTTAATTATGGCTGAATCATCCGATATTTACACGCCAACGGAGCTTCTATATTCTTTATCTGCCTGTACATACATGTGGGACGGGAATATACTAC", "Cheetah": "CTTATCAAAATCGTTAATCACTCATTCATCGATTTACCCACCCCACCTAACATTTCAGCATGATGAAACTTCGGCTCCCTACTAGGAGTCTGCCTAGTCCTACAGATCCTAACCGGCCTTTTCCTAGCCATACACTACACATCAGACACAATAACCGCCTTTTCATCAGTTACTCACATCTGCCGCGACGTCAACTACGGCTGAATTATTCGATACATGCACGCCAACGGAGCCTCTATATTCTTTATCTGCCTATACATGCATGTAGGACGAGAATATACTAC", "Puma": "CTTATCAAAATCATTAATCACTCATTTATTGATTTACCCACCCCATCCAACATTTCAGCATGATGAAACTTTGGCTCCCTACTAGGGGTCTGCCTAATCCTACAAATCCTAACCGGCCTCTTCCTGGCCATACACTATACATCAGACACAATGACTGCCTTTTCATCAGTCACTCACATCTGTCGTGACGTTAACTACGGCTGAATTATTCGGTACATACATGCCAACGGAGCCTCCATATTCTTTATCTGCCTATACATGCACGTGGGACGAGAATATATTAT", "Tiger": "CTTATCAAAATTATTAATCACTCATTTATTGACCTACCCGCCCCATCCAATATTTCAGCATGATGAAACTTTGGCTCCTTACTAGGGGTGTGCTTAATCTTACAAATCCTCACTGGCCTCTTTCTAGCCATACACTACACATCAGACACAATAACCGCATTCTCATCAGTTACCCACATTTGCCGCGACGTAAACTACGGCTGGATTATCCGATATCTACATGCCAACGGAGCCTCCATATTCTTTATCTGTCTATACATGCACGTAGGACGAGAATATACTAC", "Lion": "CTTGTCAAAATTATTAATCACTCATTCATTGATCTTCCCACTCCACCCAATATCTCAGCATGATGAAACTTTGGCTCCTTATTAGGAGTATGTTTAATCCTACAAATTCTCACCGGCCTCTTTCTAGCCATACATTACACACCAGACACAATAACCGCTTTCTCATCAGTCACCCACATTTGCCGCGATGTAAACTATGGCTGAATTATCCGGTACCTACACGCCAACGGAGCCTCCATATTCTTTATCTGCCTATACATGCATGTAGGACGAGAATATACTAT", "Leopard": "CTCATCAAAATTATTAATCACTCATTCATTGATCTCCCCGCTCCATCCAACATCTCAACATGATGGAACTTTGGCTCCCTATTAGGGGTATGTTTAATCCTACAAATTCTCACCGGCCTCTTTCTAGCCATACATTATACATCAGACACAACAACCGCTTTCTCATCAGTTACCCATATCTGCCGCGATGTAAATTATGGCTGAATTATCCGGTATCTACACGCCAATGGAGCCTCCATATTCTTTATCTGCCTATACATACATGTAGGACGAGGATATACTAT", "Dog": "CTAGCCAAAATTGTTAATAACTCATTCATTGACCTCCCAGCGCCGTCTAACATCTCTGCTTGATGGAACTTCGGATCCTTACTAGGAGTATGCTTGATTCTACAGATTCTAACAGGTTTATTCTTAGCTATGCACTATACATCGGACACAGCCACAGCTTTTTCATCAGTCACCCACATCTGCCGAGACGTTAACTACGGCTGAATTATCCGCTATATGCACGCAAATGGCGCTTCCATATTCTTTATCTGCCTATTCCTACATGTAGGACGAGCCTATATTAC", "Wolf": "CTAGCCAAAATTGTTAATAACTCATTCATTGACCTCCCAGCGCCGTCTAACATCTCTGCTTGATGGAACTTCGGATCCTTACTAGGAGTATGCTTGATTCTACAGATTCTAACAGGTTTATTCTTAGCTATGCACTATACATCGGACACAGCCACAGCTTTTTCATCAGTCACCCACATCTGCCGAGACGTTAACTACGGCTGAATTATCCGCTATATGCACGCAAATGGCGCTTCCATATTCTTTATCTGCCTATTCCTACATGTAGGACGAGCCTATATTAC", "Black": "TTAGCTAAAATCATCAACAACTCACTCATTGATCTCCCAGCACCATCAAATATCTCAGCATGATGAAACTTCGGGTCCCTCCTCGGAGTATGTTTAGTACTACAAATTCTAACGGGCCTATTCCTAGCTATACACTATACATCAGACACAACTACAGCCTTTTCATCAATCACCCATATTTGCCGAGATGTTCACTACGGATGAATTATCCGATACATACATGCTAACGGAGCTTCCATATTCTTTATCTGCCTGTTCATGCACGTAGGACGGGTCTGTACTAT", "Brown": "TTAGCTAAAATCATCAACAACTCATTTATTGACCTTCCAACACCATCAAACATCTCAGCATGATGAAACTTTGGATCCCTCCTTGGAGTATGTTTAATTCTACAGATTCTAACAGGCCTGTTCCTAGCCATACACTATACACCAGACACAACCACAGCTTTTTCATCGGTCACCCACATTTGCCGAGACGTTCACTACGGGTGAGTTATCCGATATGTACATGCAAATGGAGCCTCCATCTTCTTTATCTGCCTATTTATGCACTAGGACGGGGCCTGTACTAT", "Cave": "TTAGCTAAAATCATCAACAACTCATTTATTGACCTCCCAACACCATCAAACATCTCAGCATGATGAAACTTTGGATCCCTCCTCGGAGTATGCTTAATTCTACAGATCCTAACAGGCCTGTTTCTAGCTATACACTACACATCAGACACAACCACAGCCTTTTCATCAATCACCCATATTTGCCGAGACGTTCACTACGGTTGAGTTATCCGATATATACATGCAAACGGAGCCTCCATATTCTTTATCTGTCTATTCATGCACTAGGACGGGGCCTATACTAT"}

In [125]:
matrice =[
        [0,4,8,2],
        [4,0,8,4],
        [8,8,0,8],
        [2,4,8,0]
]

labels = ['A', 'B', 'C', 'D']
matrice_labels = list(dic_seq.keys())
#matrice_all = matrice_distance(dic_seq)
UPGMA(matrice, labels)

(A:1),(D:1):1,(B:2):2,(C:4)


: 

**Exercice 8 :** Quelles sont les hypothèses faites par UPGMA ? Semblent-elles respectées dans le cas présent ?

Réponse : 

----
## Enracinement de l'arbre

Après avoir utilisé UPGMA pour réaliser votre arbre, l'enracinement s'est normalement fait au poids moyen. 

**Exercice 9 :** Quelle autre méthode est-il possible d'utiliser pour enraciner un arbre ? Pouvez-vous l'utilisez ici ? Si oui, afficher le nouvel arbre.

**Réponse :** 

----
## Neighbor-joining

Le neighbor-joining est un autre algorithme permettant de calculer un arbre phylogénique à partir d'une matrice de distance. Il a l'avantage de faire moins d'hypothèse qu'UPGMA sur les données (elles ne sont plus forcément ultramétrique) et il donne donc de meilleurs arbres dans presque tout les cas. Vous trouverez un example d'application de cet algorithme [ici](http://www.evolution-textbook.org/content/free/tables/Ch_27/T11_EVOW_Ch27.pdf).

**Exercice 10 :** Implémenter l'algorithme du neighbor-joining, appliquer-le aux données puis enraciner l'arbre.

In [ ]:
# À remplir
def neighbor_joining(M):
    

----
## Bootstrap

Le bootstrap est une méthode permettant de calculer la robustesse des branches d'un arbre. Il sagit de recréer un jeu de données artificiel en tirant n positions dans l'alignement des séquences avec remise, n étant la longueur de l'alignement. On recréer un arbre à partir de ces nouvelles données puis on le compare avec l'arbre obtenu avec les données réelles. Si une branche de l'arbre réelle est présente dans l'arbre artificiel, son bootstrap augmente de 1. On répètera la procédure 100 fois afin d'avoir un score sur 100 pour chaque branche.

**Exercice 11 :** Calculer le bootstrap des branches des arbres obtenus avec UPGMA et neighbor-joining.

In [ ]:
# À remplir

----
## Conclusion

**Exercice 12 :** Quelles sont vos conclusions par rapport à l'arbre phylogénique de _smilodon_, _homotherium_ et _M. trumani_ ? Comparer les deux méthodes. Comment expliquer les caractéristiques morphologiques similaires entre ces trois espèces ? Une réponse détaillée est attendue.

Réponse :